In [1]:
import os, sys
# sys.path.append(os.path.abspath(os.path.join(__file__, "..", "..", "..")))
from autoPyTorch import AutoNetImageClassification
from autoPyTorch.data_management.data_manager import DataManager
import numpy as np

In [12]:
autonet = AutoNetImageClassification(config_preset="full_cs", networks=["resnet9"])
hyperparameter_config = autonet.get_hyperparameter_search_space().sample_configuration().get_dictionary()
config = autonet.get_current_autonet_config()
config['result_logger_dir'] = './results'
print(hyperparameter_config)

{'CreateImageDataLoader:batch_size': 110, 'ImageAugmentation:augment': False, 'ImageAugmentation:cutout': True, 'LossModuleSelectorIndices:loss_module': 'cross_entropy', 'NetworkSelectorDatasetInfo:network': 'resnet9', 'OptimizerSelector:optimizer': 'rmsprop', 'SimpleLearningrateSchedulerSelector:lr_scheduler': 'cyclic', 'SimpleTrainNode:batch_loss_computation_technique': 'standard', 'ImageAugmentation:cutout_holes': 3, 'ImageAugmentation:length': 17, 'NetworkSelectorDatasetInfo:resnet9:conv_bn': 'conv_pool_bn_act', 'OptimizerSelector:rmsprop:alpha': 0.7277814280937108, 'OptimizerSelector:rmsprop:learning_rate': 0.004443696398389036, 'OptimizerSelector:rmsprop:momentum': 0.42523072016205193, 'OptimizerSelector:rmsprop:weight_decay': 0.0914885764715718, 'SimpleLearningrateSchedulerSelector:cyclic:cycle_length': 4, 'SimpleLearningrateSchedulerSelector:cyclic:max_factor': 1.2273085661261378, 'SimpleLearningrateSchedulerSelector:cyclic:min_factor': 0.03860898538456489}


In [10]:
path_to_cifar_csv = os.path.abspath("datasets/CIFAR10.csv")

In [11]:
autonet.refit(X_train=np.array([path_to_cifar_csv]),
                                Y_train=np.array([0]),
                                X_valid=None,
                                Y_valid=None,
                                hyperparameter_config=hyperparameter_config,
                                autonet_config=config,
                                budget=50)

{'CreateImageDataLoader:batch_size': 110,
 'ImageAugmentation:augment': False,
 'ImageAugmentation:cutout': True,
 'ImageAugmentation:cutout_holes': 3,
 'ImageAugmentation:length': 32,
 'LossModuleSelectorIndices:loss_module': 'cross_entropy',
 'NetworkSelectorDatasetInfo:network': 'resnet9',
 'NetworkSelectorDatasetInfo:resnet9:conv_bn': 'conv_pool_bn_act',
 'OptimizerSelector:optimizer': 'rmsprop',
 'OptimizerSelector:rmsprop:alpha': 0.7277814280937108,
 'OptimizerSelector:rmsprop:learning_rate': 0.004443696398389036,
 'OptimizerSelector:rmsprop:momentum': 0.42523072016205193,
 'OptimizerSelector:rmsprop:weight_decay': 0.0914885764715718,
 'SimpleLearningrateSchedulerSelector:cosine_annealing:eta_min': 1e-08,
 'SimpleLearningrateSchedulerSelector:cyclic:cycle_length': 4,
 'SimpleLearningrateSchedulerSelector:cyclic:max_factor': 1.2273085661261378,
 'SimpleLearningrateSchedulerSelector:cyclic:min_factor': 0.03860898538456489,
 'SimpleLearningrateSchedulerSelector:lr_scheduler': 'cosin

{'loss': inf,
 'optimized_hyperparameter_config': {'NetworkSelectorDatasetInfo:network': 'resnet9',
  'SimpleLearningrateSchedulerSelector:cosine_annealing:eta_min': 1e-08,
  'CreateImageDataLoader:batch_size': 110,
  'ImageAugmentation:augment': False,
  'ImageAugmentation:cutout': True,
  'LossModuleSelectorIndices:loss_module': 'cross_entropy',
  'OptimizerSelector:optimizer': 'rmsprop',
  'SimpleLearningrateSchedulerSelector:lr_scheduler': 'cosine_annealing',
  'SimpleTrainNode:batch_loss_computation_technique': 'standard',
  'ImageAugmentation:cutout_holes': 3,
  'ImageAugmentation:length': 32,
  'NetworkSelectorDatasetInfo:resnet9:conv_bn': 'conv_pool_bn_act',
  'OptimizerSelector:rmsprop:alpha': 0.7277814280937108,
  'OptimizerSelector:rmsprop:learning_rate': 0.004443696398389036,
  'OptimizerSelector:rmsprop:momentum': 0.42523072016205193,
  'OptimizerSelector:rmsprop:weight_decay': 0.0914885764715718,
  'SimpleLearningrateSchedulerSelector:cyclic:cycle_length': 4,
  'SimpleLea